In [2]:

from pathlib import Path

import pandas as pd
import numpy as np

import biopsykit as bp


from biopsykit.classification.model_selection import SklearnPipelinePermuter

from pepbench.io import load_best_performing_algos_b_point, load_preprocessed_training_data, compute_abs_error, \
     get_best_pipeline_results, get_best_estimator, get_pipeline_steps, compute_mae_std_from_permuter

import pingouin as pg

import matplotlib.pyplot as plt

#%matplotlib widget
%load_ext autoreload
%autoreload 2

In [110]:
model_path = Path(f"../../../results/models")
data_path = Path(f"../../../results/data")
result_path = Path(f"../../../results")

Analyse the parameters of the SS_KBest_SVR_RR-Interval-Median-Imputed model

In [124]:
ss_kbest_svr_rr_interval_median_imputed = SklearnPipelinePermuter.from_pickle(model_path.joinpath('q-peak/rr-interval/q_peak_regression_hpc_0_baseline_result_rr_interval_median_imputed.pkl'))

In [135]:
ss_kbest_svr_rr_interval_median_imputed_summary = ss_kbest_svr_rr_interval_median_imputed.metric_summary()
ss_kbest_svr_rr_interval_median_imputed_summary[ss_kbest_svr_rr_interval_median_imputed_summary.index == ('StandardScaler', 'SelectKBest', 'SVR')]

,,,conf_matrix,conf_matrix_folds,true_labels,true_labels_folds,predicted_labels,predicted_labels_folds,train_indices,train_indices_folds,test_indices,test_indices_folds,mean_test_neg_mean_absolute_error,std_test_neg_mean_absolute_error,test_neg_mean_absolute_error_fold_0,test_neg_mean_absolute_error_fold_1,test_neg_mean_absolute_error_fold_2,test_neg_mean_absolute_error_fold_3,test_neg_mean_absolute_error_fold_4
pipeline_scaler,pipeline_reduce_dim,pipeline_clf,,,,,,,,,,,,,,,,,
StandardScaler,SelectKBest,SVR,[0.0],[],"[302.0, 300.0, 308.0, 312.0, 294.0, 298.0, 292...","[[302.0, 300.0, 308.0, 312.0, 294.0, 298.0, 29...","[301.6734171879768, 299.8199721121548, 307.877...","[[301.6734171879768, 299.8199721121548, 307.87...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13...","[3180, 3181, 3182, 3183, 3184, 3185, 3186, 318...","[[3180, 3181, 3182, 3183, 3184, 3185, 3186, 31...",2.429628,0.409388,2.267894,2.967666,2.202106,2.834568,1.875905


In [127]:
pipeline_svr_q_peak = ss_kbest_svr_rr_interval_median_imputed.best_estimator_summary().loc[('StandardScaler', 'SelectFromModel', 'SVR')]['best_estimator'].pipeline
pipeline_svr_q_peak

[Pipeline(memory=Memory(location=cachedir/joblib),
          steps=[('scaler', StandardScaler()),
                 ('reduce_dim',
                  SelectFromModel(estimator=RandomForestRegressor(random_state=0),
                                  threshold='median')),
                 ('clf',
                  SVR(C=np.float64(100.0), epsilon=np.float64(0.01),
                      gamma=np.float64(1.0)))]),
 Pipeline(memory=Memory(location=cachedir/joblib),
          steps=[('scaler', StandardScaler()),
                 ('reduce_dim',
                  SelectFromModel(estimator=RandomForestRegressor(random_state=0),
                                  threshold='median')),
                 ('clf',
                  SVR(C=np.float64(100.0),
                      epsilon=np.float64(0.09999999999999999),
                      gamma=np.float64(1.0)))]),
 Pipeline(memory=Memory(location=cachedir/joblib),
          steps=[('scaler', StandardScaler()),
                 ('reduce_dim',
         

In [130]:
features = np.array(pd.read_csv(data_path.joinpath('q-peak/rr-interval/train_data_q_peak_rr_interval_median_imputed.csv'), index_col=[0,1,2,3,4,5]).columns)
features

array(['rr_interval_ms_estimated', 'forounzafar2018', 'martinez2004',
       'vanlien2013-32-ms', 'vanlien2013-34-ms', 'vanlien2013-36-ms',
       'vanlien2013-38-ms', 'vanlien2013-40-ms', 'vanlien2013-42-ms'],
      dtype=object)

In [133]:
all_selected_features = []
for fold in range(0,5,1):
    selected_features = features[pipeline_svr_q_peak[fold]['reduce_dim'].get_support()]
    all_selected_features.append(selected_features)
all_selected_features

[array(['forounzafar2018', 'vanlien2013-32-ms', 'vanlien2013-36-ms',
        'vanlien2013-40-ms', 'vanlien2013-42-ms'], dtype=object),
 array(['forounzafar2018', 'martinez2004', 'vanlien2013-32-ms',
        'vanlien2013-34-ms', 'vanlien2013-38-ms'], dtype=object),
 array(['forounzafar2018', 'vanlien2013-32-ms', 'vanlien2013-34-ms',
        'vanlien2013-36-ms', 'vanlien2013-38-ms'], dtype=object),
 array(['forounzafar2018', 'vanlien2013-32-ms', 'vanlien2013-34-ms',
        'vanlien2013-36-ms', 'vanlien2013-42-ms'], dtype=object),
 array(['forounzafar2018', 'vanlien2013-32-ms', 'vanlien2013-34-ms',
        'vanlien2013-38-ms', 'vanlien2013-40-ms'], dtype=object)]

Analyse the parameters of the SS_SFM_SVR_RR-Interval model

In [111]:
ss_sfm_svr_rr_interval = SklearnPipelinePermuter.from_pickle(model_path.joinpath('b-point/rr-interval/b_point_regression_hpc_0_baseline_result_rr_interval.pkl'))

In [136]:
ss_sfm_svr_rr_interval_summary = ss_sfm_svr_rr_interval.metric_summary()
ss_sfm_svr_rr_interval_summary[ss_sfm_svr_rr_interval_summary.index == ('StandardScaler', 'SelectFromModel', 'SVR')]

,,,conf_matrix,conf_matrix_folds,true_labels,true_labels_folds,predicted_labels,predicted_labels_folds,train_indices,train_indices_folds,test_indices,test_indices_folds,mean_test_neg_mean_absolute_error,std_test_neg_mean_absolute_error,test_neg_mean_absolute_error_fold_0,test_neg_mean_absolute_error_fold_1,test_neg_mean_absolute_error_fold_2,test_neg_mean_absolute_error_fold_3,test_neg_mean_absolute_error_fold_4
pipeline_scaler,pipeline_reduce_dim,pipeline_clf,,,,,,,,,,,,,,,,,
StandardScaler,SelectFromModel,SVR,[0.0],[],"[370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...","[[370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 33...","[375.819959280204, 350.1890294618498, 374.0464...","[[375.819959280204, 350.1890294618498, 374.046...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13...","[2241, 2242, 2243, 2244, 2245, 2246, 2247, 224...","[[2241, 2242, 2243, 2244, 2245, 2246, 2247, 22...",7.631287,0.7136,7.908656,6.37793,7.504899,7.811871,8.553078


In [114]:
pipeline_svr = ss_sfm_svr_rr_interval.best_estimator_summary().loc[('StandardScaler', 'SelectFromModel', 'SVR')]['best_estimator'].pipeline
pipeline_svr

[Pipeline(memory=Memory(location=cachedir/joblib),
          steps=[('scaler', StandardScaler()),
                 ('reduce_dim',
                  SelectFromModel(estimator=RandomForestRegressor(random_state=0),
                                  threshold='median')),
                 ('clf',
                  SVR(C=np.float64(100.0),
                      epsilon=np.float64(0.09999999999999999),
                      gamma=np.float64(1.0)))]),
 Pipeline(memory=Memory(location=cachedir/joblib),
          steps=[('scaler', StandardScaler()),
                 ('reduce_dim',
                  SelectFromModel(estimator=RandomForestRegressor(random_state=0),
                                  threshold='median')),
                 ('clf',
                  SVR(C=np.float64(100.0), epsilon=np.float64(1.0),
                      gamma=np.float64(1.0)))]),
 Pipeline(memory=Memory(location=cachedir/joblib),
          steps=[('scaler', StandardScaler()),
                 ('reduce_dim',
          

In [115]:
features = np.array(pd.read_csv(data_path.joinpath('b-point/rr-interval/train_data_b_point_rr_interval.csv'), index_col=[0,1,2,3,4,5]).columns)
features

array(['rr_interval_ms_estimated', 'arbol2017-isoelectric-crossings',
       'arbol2017-second-derivative', 'arbol2017-third-derivative',
       'debski1993-second-derivative', 'drost2022', 'forounzafar2018',
       'lozano2007-linear-regression', 'lozano2007-quadratic-regression',
       'sherwood1990', 'stern1985'], dtype=object)

In [116]:
all_selected_features = []
for fold in range(0,5,1):
    selected_features = features[pipeline_svr[fold]['reduce_dim'].get_support()]
    all_selected_features.append(selected_features)
all_selected_features

[array(['debski1993-second-derivative', 'drost2022', 'forounzafar2018',
        'lozano2007-linear-regression', 'lozano2007-quadratic-regression',
        'stern1985'], dtype=object),
 array(['debski1993-second-derivative', 'drost2022', 'forounzafar2018',
        'lozano2007-linear-regression', 'lozano2007-quadratic-regression',
        'stern1985'], dtype=object),
 array(['debski1993-second-derivative', 'drost2022', 'forounzafar2018',
        'lozano2007-linear-regression', 'lozano2007-quadratic-regression',
        'stern1985'], dtype=object),
 array(['debski1993-second-derivative', 'drost2022', 'forounzafar2018',
        'lozano2007-linear-regression', 'lozano2007-quadratic-regression',
        'stern1985'], dtype=object),
 array(['debski1993-second-derivative', 'drost2022', 'forounzafar2018',
        'lozano2007-linear-regression', 'lozano2007-quadratic-regression',
        'stern1985'], dtype=object)]

Analyse the parameters of the MinMax_KBest_RFR_RR-Interval-Median-Imputed model

In [117]:
minmax_kbest_rfr_rr_interval_median_imputed = SklearnPipelinePermuter.from_pickle(model_path.joinpath('b-point/rr-interval/b_point_regression_hpc_0_baseline_result_rr_interval_median_imputed.pkl'))

In [ ]:
minmax_kbest_rfr_rr_interval_median_imputed_summary = minmax_kbest_rfr_rr_interval_median_imputed.metric_summary()
minmax_kbest_rfr_rr_interval_median_imputed_summary[minmax_kbest_rfr_rr_interval_median_imputed_summary.index == ('StandardScaler', 'SelectFromModel', 'RandomForestRegressor')]

,,,conf_matrix,conf_matrix_folds,true_labels,true_labels_folds,predicted_labels,predicted_labels_folds,train_indices,train_indices_folds,test_indices,test_indices_folds,mean_test_neg_mean_absolute_error,std_test_neg_mean_absolute_error,test_neg_mean_absolute_error_fold_0,test_neg_mean_absolute_error_fold_1,test_neg_mean_absolute_error_fold_2,test_neg_mean_absolute_error_fold_3,test_neg_mean_absolute_error_fold_4
pipeline_scaler,pipeline_reduce_dim,pipeline_clf,,,,,,,,,,,,,,,,,
StandardScaler,SelectFromModel,RandomForestRegressor,[0.0],[],"[516.0, 520.0, 492.0, 492.0, 498.0, 334.0, 516...","[[516.0, 520.0, 492.0, 492.0, 498.0, 334.0, 51...","[527.9756659500838, 520.1517644170993, 491.065...","[[527.9756659500838, 520.1517644170993, 491.06...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13...","[318, 319, 320, 321, 322, 323, 324, 325, 326, ...","[[318, 319, 320, 321, 322, 323, 324, 325, 326,...",8.378214,1.016892,7.581512,8.360566,9.379543,6.958986,9.610462


In [120]:
pipeline_rfr = minmax_kbest_rfr_rr_interval_median_imputed.best_estimator_summary().loc[('StandardScaler', 'SelectFromModel', 'RandomForestRegressor')]['best_estimator'].pipeline
pipeline_rfr

[Pipeline(memory=Memory(location=cachedir/joblib),
          steps=[('scaler', StandardScaler()),
                 ('reduce_dim',
                  SelectFromModel(estimator=RandomForestRegressor(random_state=0),
                                  threshold='median')),
                 ('clf',
                  RandomForestRegressor(ccp_alpha=0.01, max_depth=np.int64(70),
                                        max_features=0.5, min_samples_split=10,
                                        n_estimators=400,
                                        random_state=RandomState(MT19937) at 0x1D9FD2A1940))]),
 Pipeline(memory=Memory(location=cachedir/joblib),
          steps=[('scaler', StandardScaler()),
                 ('reduce_dim',
                  SelectFromModel(estimator=RandomForestRegressor(random_state=0),
                                  threshold='median')),
                 ('clf',
                  RandomForestRegressor(criterion='friedman_mse',
                                

In [121]:
features = np.array(pd.read_csv(data_path.joinpath('b-point/rr-interval/train_data_b_point_rr_interval_median_imputed.csv'), index_col=[0,1,2,3,4, 5]).columns)
features

array(['rr_interval_ms_estimated', 'arbol2017-isoelectric-crossings',
       'arbol2017-second-derivative', 'arbol2017-third-derivative',
       'debski1993-second-derivative', 'drost2022', 'forounzafar2018',
       'lozano2007-linear-regression', 'lozano2007-quadratic-regression',
       'sherwood1990', 'stern1985'], dtype=object)

In [122]:
all_feature_importances = []
for fold in range(0,5,1):
    selected_features = features[pipeline_rfr[fold]['reduce_dim'].get_support()]
    feature_importances = pipeline_rfr[fold]['clf'].feature_importances_
    all_feature_importances.append(list(zip(selected_features, feature_importances)))
all_feature_importances = [
    [(key, float(value)) for key, value in inner_list]
    for inner_list in all_feature_importances
]
all_feature_importances

[[('rr_interval_ms_estimated', 0.013369264018898024),
  ('debski1993-second-derivative', 0.1428500944126103),
  ('drost2022', 0.5044008514348968),
  ('forounzafar2018', 0.07159688439220195),
  ('lozano2007-linear-regression', 0.25423962483986645),
  ('stern1985', 0.013543280901526341)],
 [('rr_interval_ms_estimated', 0.013520086499489499),
  ('debski1993-second-derivative', 0.11118687497830865),
  ('drost2022', 0.5072016842520183),
  ('forounzafar2018', 0.08214358936728436),
  ('lozano2007-linear-regression', 0.2718950629071309),
  ('stern1985', 0.014052701995768295)],
 [('rr_interval_ms_estimated', 0.034237967175627375),
  ('debski1993-second-derivative', 0.06585057783143003),
  ('drost2022', 0.6540674286894496),
  ('forounzafar2018', 0.016585293818113505),
  ('lozano2007-linear-regression', 0.2217830188693492),
  ('stern1985', 0.007475713616030409)],
 [('rr_interval_ms_estimated', 0.015427152572372687),
  ('debski1993-second-derivative', 0.10437341729240959),
  ('drost2022', 0.502463